In [ ]:
# import sys
# !conda install -c conda-forge --yes --prefix {sys.prefix} matplotlib

## Let the games begin

In [ ]:
import sys
sys.path.insert(0, '..')

In [ ]:
from config.path_config import GraphPaths

### Facebook Dataset

In [ ]:
import networkx as nx
import pandas as pd
import os

In [ ]:
df = pd.read_csv(GraphPaths.fb_links, sep='\t', header=None, usecols=[0, 1])

In [ ]:
df

In [ ]:
fbG = nx.from_pandas_edgelist(df, source=0, target=1)

In [ ]:
print('nodes: ', fbG.number_of_nodes(), ', edges: ', fbG.number_of_edges())

In [ ]:
import random
random.seed(42)

In [ ]:
def generate_separate_sampled_graphs(og_graph, alpha_s, alpha_c):
    subgraph1 = nx.Graph()
    subgraph2 = nx.Graph()
        
    for edge in list(og_graph.edges()):
        p = random.uniform(0, 1)
        if p < (1 - (2 * alpha_s) + (alpha_s * alpha_c)):
            continue
        elif (1 - (2 * alpha_s) + (alpha_s * alpha_c)) < p < (1 - alpha_s):
            subgraph1.add_edge(*edge, weight=1)
        elif (1 - alpha_s) < p < (1 - (alpha_s * alpha_c)):
            subgraph2.add_edge(*edge, weight=1)
        else:
            subgraph1.add_edge(*edge, weight=1)
            subgraph2.add_edge(*edge, weight=1)
    return subgraph1, subgraph2

In [ ]:
def filter_nodes_by_degree(og_graph, min_degree):
    ctr = 0
    filtered_graph = nx.Graph()
    for edge in og_graph.edges():
        if og_graph.degree(edge[0]) > min_degree and og_graph.degree(edge[1]) > min_degree:
            filtered_graph.add_edge(*edge, weight=1)
    return filtered_graph

In [ ]:
filtered_fbG = filter_nodes_by_degree(fbG, 5)

In [ ]:
print('nodes: ', filtered_fbG.number_of_nodes(), ', edges: ', filtered_fbG.number_of_edges())

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_bar(degree_list, path=None, xscale=None):
    index = np.arange(len(degree_list))
    plt.bar(index, degree_list)
    if xscale:
        plt.xscale(xscale)
    plt.xlabel('Degree')
    plt.ylabel('Count')
    plt.title('Degree Distribution')
    plt.grid(True)
    if path:
        plt.savefig(path, dpi=300, bbox_inches='tight')
    plt.show()

In [ ]:
deg_hist = nx.degree_histogram(filtered_fbG)

In [ ]:
plot_bar(deg_hist)

In [ ]:
plot_bar(deg_hist, xscale='log')

In [ ]:
plot_bar(deg_hist[:10])

In [ ]:
alpha_s = 0.5
alpha_c = 0.5

fbG1, fbG2 = generate_separate_sampled_graphs(filtered_fbG, alpha_s, alpha_c)

In [ ]:
print('nodes: ', fbG1.number_of_nodes(), ', edges: ', fbG1.number_of_edges())
print('nodes: ', fbG2.number_of_nodes(), ', edges: ', fbG2.number_of_edges())

In [ ]:
u_suffix = lambda label : str(label) + '-u'
v_suffix = lambda label : str(label) + '-v'

In [ ]:
relabeled_fbG1 = nx.relabel_nodes(fbG1, u_suffix)
relabeled_fbG2 = nx.relabel_nodes(fbG2, v_suffix)

In [ ]:
out_dir = os.path.dirname(GraphPaths.fb_subgraph.format('1', alpha_s, alpha_c))
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
nx.write_edgelist(relabeled_fbG1, GraphPaths.fb_subgraph.format('1', alpha_s, alpha_c), data=['weight'])
nx.write_edgelist(relabeled_fbG2, GraphPaths.fb_subgraph.format('2', alpha_s, alpha_c), data=['weight'])